<a href="https://colab.research.google.com/github/LuluBeatson/privacy/blob/master/Membership_inference_codelab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2020 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Assess privacy risks with TensorFlow Privacy Membership Inference Attacks

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/privacy/blob/master/tensorflow_privacy/privacy/membership_inference_attack/codelab.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/privacy/blob/master/tensorflow_privacy/privacy/membership_inference_attack/codelab.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

##Overview
In this codelab we'll train a simple image classification model on the CIFAR10 dataset, and then use the "membership inference attack" against this model to assess if the attacker is able to "guess" whether a particular sample was present in the training set.

## Setup
First, set this notebook's runtime to use a GPU, under Runtime > Change runtime type > Hardware accelerator. Then, begin importing the necessary libraries.

In [ ]:
#@title Import statements.
import numpy as np
from typing import Tuple, Text
from scipy import special

import tensorflow as tf
import tensorflow_datasets as tfds

# Set verbosity.
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter(action="ignore", category=ConvergenceWarning)
simplefilter(action="ignore", category=FutureWarning)

Install TensorFlow Privacy.

In [ ]:
!pip3 install git+https://github.com/tensorflow/privacy

from tensorflow_privacy.privacy.membership_inference_attack import membership_inference_attack as mia

  Cloning https://github.com/tensorflow/privacy to /tmp/pip-req-build-57zfjr__
  Running command git clone -q https://github.com/tensorflow/privacy /tmp/pip-req-build-57zfjr__
  Created wheel for tensorflow-privacy: filename=tensorflow_privacy-0.5.0-cp36-none-any.whl size=146934 sha256=1fbe20aa635bd2a4956004d25f6c6b1084ed941fd73b2b43ace01247120aadb5
  Stored in directory: /tmp/pip-ephem-wheel-cache-80wsi4xf/wheels/8a/e4/14/41d16468ac11ec804bd21cfb75fc2e24f96b9e4c5af778f576
Successfully built tensorflow-privacy
  Found existing installation: tensorflow-privacy 0.2.2
    Uninstalling tensorflow-privacy-0.2.2:
      Successfully uninstalled tensorflow-privacy-0.2.2


## Train a simple model on CIFAR10 with Keras.

In [ ]:
dataset = 'cifar10'
num_classes = 10
num_conv = 3
activation = 'relu'
optimizer = 'adam'
lr = 0.02
momentum = 0.9
batch_size = 250
epochs = 100  # Privacy risks are especially visible with lots of epochs.


def small_cnn(input_shape: Tuple[int],
              num_classes: int,
              num_conv: int,
              activation: Text = 'relu') -> tf.keras.models.Sequential:
  """Setup a small CNN for image classification.

  Args:
    input_shape: Integer tuple for the shape of the images.
    num_classes: Number of prediction classes.
    num_conv: Number of convolutional layers.
    activation: The activation function to use for conv and dense layers.

  Returns:
    The Keras model.
  """
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Input(shape=input_shape))

  # Conv layers
  for _ in range(num_conv):
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation=activation))
    model.add(tf.keras.layers.MaxPooling2D())

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(64, activation=activation))
  model.add(tf.keras.layers.Dense(num_classes))
  return model


print('Loading the dataset.')
train_ds = tfds.as_numpy(
    tfds.load(dataset, split=tfds.Split.TRAIN, batch_size=-1))
test_ds = tfds.as_numpy(
    tfds.load(dataset, split=tfds.Split.TEST, batch_size=-1))
x_train = train_ds['image'].astype('float32') / 255.
y_train_indices = train_ds['label'][:, np.newaxis]
x_test = test_ds['image'].astype('float32') / 255.
y_test_indices = test_ds['label'][:, np.newaxis]

# Convert class vectors to binary class matrices.
y_train = tf.keras.utils.to_categorical(y_train_indices, num_classes)
y_test = tf.keras.utils.to_categorical(y_test_indices, num_classes)

input_shape = x_train.shape[1:]

model = small_cnn(
    input_shape, num_classes, num_conv=num_conv, activation=activation)

print('Optimizer ', optimizer)
print('learning rate %f', lr)

optimizer = tf.keras.optimizers.SGD(lr=lr, momentum=momentum)

loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
model.summary()
model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(x_test, y_test),
    shuffle=True)
print('Finished training.')

Loading the dataset.


/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.0.incomplete37K7UE/cifar10-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.0.incomplete37K7UE/cifar10-test.tfrecord


Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.0. Subsequent calls will reuse this data.
Optimizer  adam
learning rate %f 0.02
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 32)          9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling

In [ ]:
#@title Calculate logits, probabilities and loss values for training and test sets.
#@markdown We will use these values later in the membership inference attack to
#@markdown separate training and test samples.
print('Predict on train...')
logits_train = model.predict(x_train, batch_size=batch_size)
print('Predict on test...')
logits_test = model.predict(x_test, batch_size=batch_size)

print('Apply softmax to get probabilities from logits...')
prob_train = special.softmax(logits_train)
prob_test = special.softmax(logits_test)

print('Compute losses...')
cce = tf.keras.backend.categorical_crossentropy
constant = tf.keras.backend.constant

loss_train = cce(constant(y_train), constant(prob_train), from_logits=False).numpy()
loss_test = cce(constant(y_test), constant(prob_test), from_logits=False).numpy()

## Run membership inference attacks.

In [ ]:
#@markdown We will now execute membership inference attack against the
#@markdown previously trained CIFAR10 model. This will generate a number of
#@markdown scores (most notably, attacker advantage and AUC for the membership
#@markdown inference classifier). An AUC of close to 0.5 means that the attack
#@markdown isn't able to identify training samples, which means that the model
#@markdown doesn't have privacy issues according to this test. Higher values,
#@markdown on the contrary, indicate potential privacy issues.

labels_train = np.argmax(y_train, axis=1)
labels_test = np.argmax(y_test, axis=1)

results_without_classifiers = mia.run_all_attacks(
    loss_train,
    loss_test,
    logits_train,
    logits_test,
    labels_train,
    labels_test,
    attack_classifiers=[],
)
print(results_without_classifiers)

# Note: This will take a while, since it also trains ML models to
# separate train/test examples. If it's taking too looking, use
# the `run_all_attacks` function instead.
attack_result_summary = mia.run_all_attacks_and_create_summary(
    loss_train,
    loss_test,
    logits_train,
    logits_test,
    labels_train,
    labels_test,
)[0]

print(attack_result_summary)

This is the end of the codelab! Feel free to change the parameters to see how the privacy risks change.